In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
function_df = pd.read_csv('TEM-1/TEM-1.txt')

In [21]:
function_df.head()

,protein,uniprot,dms_id,position,aa1,aa2,predicted?,predictions,sequence
0,TEM-1,P62593,beta-lactamase,124,A,F,NO,-0.008031,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
1,TEM-1,P62593,beta-lactamase,124,A,H,NO,-0.008031,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
2,TEM-1,P62593,beta-lactamase,124,A,P,NO,-0.008031,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
3,TEM-1,P62593,beta-lactamase,124,A,Y,NO,-0.008031,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
4,TEM-1,P62593,beta-lactamase,132,A,K,NO,-0.008031,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...


In [30]:
function_df['sequence_len'] = function_df['sequence'].apply(lambda seq: len(seq)) 
function_df = function_df[function_df['sequence_len'] == 286]
max_len = function_df['sequence_len'].max()

In [31]:
def sequence_to_aa(seq):
  return list(seq)

In [32]:
def get_sequence_aa(df):
  df['sequence_aa'] = df['sequence'].apply(lambda seq:sequence_to_aa(seq))
  df['length'] = df['sequence'].apply(lambda seq:len(seq))
  sequence_aa = np.array(df['sequence_aa'])
  return sequence_aa

In [33]:
sequence_aa = get_sequence_aa(function_df)

In [34]:
# import neccesary tools from Keras
import keras
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequence_aa)
encoded = tokenizer.texts_to_sequences(sequence_aa)
vocab_len = len(tokenizer.word_index) + 1

In [36]:
X = np.array(encoded)
y = function_df['predictions']

from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.1)

In [37]:
X_train.shape

(3905, 286)

In [38]:
y_train.mean()

0.7508347657201031

In [39]:
from keras.layers import Input, Dense, Embedding, LSTM, Bidirectional, Dropout, BatchNormalization, Softmax, multiply, Lambda, Flatten, Activation, RepeatVector, Permute, LeakyReLU
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from keras.optimizers import Adam, RMSprop
from keras.initializers import Constant

# from fit_one_cycle.clr import LRFinder

In [48]:
LSTM_size = 10
dense_sizes = [16]
LSTM_dropout = 0.0
n_epochs = 1000
adam_lr = [0.003]
initializers = ['he_uniform']

In [41]:
def get_model_LSTM(initializer, LSTM_size):
    input = Input(shape = [max_len])
    embedding = Embedding(input_dim = vocab_len, 
                          output_dim = 20, 
                          input_length = max_len, 
                          trainable = True)(input)

    activations = LSTM(LSTM_size,
                       dropout = LSTM_dropout,
                       kernel_initializer = initializer)(embedding)

    sequence_representation = Dense(64, kernel_initializer = initializer)(activations)
    sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)
    sequence_representation = Dense(32, kernel_initializer = initializer)(sequence_representation)
    sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)

    output = Dense(1, bias_initializer = Constant(y_train.mean()))(sequence_representation)

    model = Model(inputs = [input], outputs = output)
    return model

In [42]:
def get_model_LSTM_attention(initializer, LSTM_size):
    input = Input(shape = [max_len])
    embedding = Embedding(input_dim = vocab_len, 
                          output_dim = 20, 
                          input_length = max_len, 
                          trainable = True)(input)

    activations = LSTM(LSTM_size, return_sequences = True,
                       dropout = LSTM_dropout,
                       kernel_initializer = initializer)(embedding)

    attention = Dense(1, activation = 'tanh', kernel_initializer = initializer)(activations)
    attention = Flatten()(attention)
    attention = Activation('softmax', name = 'attention_activations')(attention)
    attention = RepeatVector(LSTM_size)(attention)
    attention = Permute([2, 1])(attention)

    sequence_representation = multiply([activations, attention])
    sequence_representation = Lambda(lambda xin: K.sum(xin, axis = -2), output_shape = (LSTM_size,))(sequence_representation)
    
    sequence_representation = Dense(64, kernel_initializer = initializer)(sequence_representation)
    sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)
    sequence_representation = Dense(32, kernel_initializer = initializer)(sequence_representation)
    sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)

    output = Dense(1, bias_initializer = Constant(y_train.mean()))(sequence_representation)

    model = Model(inputs = [input], outputs = output)
    return model

In [43]:
def get_model_biLSTM_attention(initializer, LSTM_size):
    input = Input(shape = [max_len])
    embedding = Embedding(input_dim = vocab_len, 
                          output_dim = 20, 
                          input_length = max_len, 
                          trainable = True)(input)

    activations = Bidirectional(LSTM(LSTM_size, return_sequences = True,
                       dropout = LSTM_dropout,
                       kernel_initializer = initializer))(embedding)

    attention = Dense(1, activation = 'tanh', kernel_initializer = initializer)(activations)
    attention = Flatten()(attention)
    attention = Activation('softmax', name = 'attention_activations')(attention)
    attention = RepeatVector(LSTM_size * 2)(attention)
    attention = Permute([2, 1])(attention)

    sequence_representation = multiply([activations, attention])
    sequence_representation = Lambda(lambda xin: K.sum(xin, axis = -2), output_shape = (LSTM_size * 2,))(sequence_representation)
    
    sequence_representation = Dense(64, kernel_initializer = initializer)(sequence_representation)
    sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)
    sequence_representation = Dense(32, kernel_initializer = initializer)(sequence_representation)
    sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)

    output = Dense(1, bias_initializer = Constant(y_train.mean()))(sequence_representation)

    model = Model(inputs = [input], outputs = output)
    return model

In [ ]:
for i in range(len(adam_lr)):
    lr = adam_lr[i]
    for j in range(len(initializers)):
        initializer = initializers[j]
        print(f'adam_lr - {lr} initializer - {initializer}')

        optimizer = Adam(lr = lr)
        model = get_model_biLSTM_attention(initializer, LSTM_size)
        model.compile(optimizer = optimizer,
                     loss = 'mean_squared_error')
        # early_stopping = EarlyStopping(patience = 300)

        folder_path = 'TEM-1/model_and_checkpoints/' + f'biLSTM attention LSTM size - {LSTM_size} epochs - {n_epochs} adam_lr {lr} initializer - {initializer}'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        checkpointing = ModelCheckpoint(folder_path + '/weights.{epoch:02d}-{val_loss:.5f}.hdf5',
                               save_best_only = True)
        history = model.fit(X_train, y_train, validation_data = (X_val, y_val), 
                            batch_size = 512, epochs = n_epochs, verbose = 1,
                            callbacks = [checkpointing])

# save the model summary
#         model_json = model.to_json()
#         with open(folder_path + '.json', 'w') as file:
#             file.write(model_json)
# get the Pearson score
# y_pred = model.predict(X_test)
# y_pred = np.reshape(y_pred, (1094,))

# from scipy.stats import pearsonr, spearmanr
# pearson_score = pearsonr(y_test, y_pred)
# print(f'Pearson correlation - {pearson_score}')
# spearman_score = spearmanr(y_test, y_pred)
# print(f'Spearman correlation - {spearman_score}')

# import pickle
# pickle.dump(pearson_score, open(f'LSTM units - {LSTM_units} epochs - {n_epochs} adam_lr {adam_lr} v3.p', 'wb'))

# visualize the training and validation loss curves
        plt.figure()
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epoch = np.arange(n_epochs)
        plt.plot(epoch, loss, label = 'loss')
        plt.plot(epoch, val_loss, label = 'val_loss')
        plt.title(f'biLSTM attention LSTM size - {LSTM_size} LSTM dropout - {LSTM_dropout} epochs - {n_epochs} adam_lr {lr} initializer - {initializer}')
        plt.legend()
        plt.savefig('TEM-1/' + f'biLSTM attention LSTM size - {LSTM_size} LSTM dropout - {LSTM_dropout} epochs - {n_epochs} adam_lr {lr} initializer - {initializer}' + '.png')

adam_lr - 0.003 initializer - he_uniform
Train on 3905 samples, validate on 434 samples
Epoch 1/1000
3905/3905 [==============================] - 11s 3ms/step - loss: 0.0983 - val_loss: 0.0936
Epoch 2/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0977 - val_loss: 0.0941
Epoch 3/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0977 - val_loss: 0.0933
Epoch 4/1000
3905/3905 [==============================] - 8s 2ms/step - loss: 0.0977 - val_loss: 0.0932
Epoch 5/1000
3905/3905 [==============================] - 11s 3ms/step - loss: 0.0980 - val_loss: 0.0938
Epoch 6/1000
3905/3905 [==============================] - 8s 2ms/step - loss: 0.0978 - val_loss: 0.0936
Epoch 7/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0976 - val_loss: 0.0937
Epoch 8/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0976 - val_loss: 0.0934
Epoch 9/1000
3905/3905 [==============================] - 4s 1ms/step - loss: 

3905/3905 [==============================] - 6s 1ms/step - loss: 0.0975 - val_loss: 0.0941
Epoch 79/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0976 - val_loss: 0.0934
Epoch 80/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0976 - val_loss: 0.0933
Epoch 81/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0975 - val_loss: 0.0937
Epoch 82/1000
3905/3905 [==============================] - 7s 2ms/step - loss: 0.0975 - val_loss: 0.0934
Epoch 83/1000
3905/3905 [==============================] - 8s 2ms/step - loss: 0.0975 - val_loss: 0.0943
Epoch 84/1000
3905/3905 [==============================] - 8s 2ms/step - loss: 0.0976 - val_loss: 0.0934
Epoch 85/1000
3905/3905 [==============================] - 8s 2ms/step - loss: 0.0975 - val_loss: 0.0935
Epoch 86/1000
3905/3905 [==============================] - 7s 2ms/step - loss: 0.0975 - val_loss: 0.0935
Epoch 87/1000
3905/3905 [==============================] - 8s 2ms/ste

3905/3905 [==============================] - 6s 1ms/step - loss: 0.0960 - val_loss: 0.0948
Epoch 156/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0948 - val_loss: 0.0925
Epoch 157/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.1009 - val_loss: 0.0995
Epoch 158/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.1002 - val_loss: 0.0932
Epoch 159/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0981 - val_loss: 0.0933
Epoch 160/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0976 - val_loss: 0.0933
Epoch 161/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0976 - val_loss: 0.0939
Epoch 162/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0974 - val_loss: 0.0930
Epoch 163/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0973 - val_loss: 0.0938
Epoch 164/1000
3905/3905 [==============================] - 6

3905/3905 [==============================] - 6s 2ms/step - loss: 0.0840 - val_loss: 0.0910
Epoch 233/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0829 - val_loss: 0.0898
Epoch 234/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0827 - val_loss: 0.0899
Epoch 235/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0823 - val_loss: 0.0920
Epoch 236/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0824 - val_loss: 0.0902
Epoch 237/1000
3905/3905 [==============================] - 6s 2ms/step - loss: 0.0829 - val_loss: 0.1022
Epoch 238/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0920 - val_loss: 0.0983
Epoch 239/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0909 - val_loss: 0.0887
Epoch 240/1000
3905/3905 [==============================] - 6s 1ms/step - loss: 0.0871 - val_loss: 0.0884
Epoch 241/1000
3905/3905 [==============================] - 6

3905/3905 [==============================] - 5s 1ms/step - loss: 0.0808 - val_loss: 0.0893
Epoch 310/1000
3905/3905 [==============================] - 5s 1ms/step - loss: 0.0802 - val_loss: 0.0863
Epoch 311/1000
3905/3905 [==============================] - 5s 1ms/step - loss: 0.0775 - val_loss: 0.0858
Epoch 312/1000
3905/3905 [==============================] - 5s 1ms/step - loss: 0.0761 - val_loss: 0.0864
Epoch 313/1000
3905/3905 [==============================] - 5s 1ms/step - loss: 0.0761 - val_loss: 0.0866
Epoch 314/1000
3905/3905 [==============================] - 5s 1ms/step - loss: 0.0758 - val_loss: 0.0856
Epoch 315/1000
3905/3905 [==============================] - 5s 1ms/step - loss: 0.0752 - val_loss: 0.0879
Epoch 316/1000
3905/3905 [==============================] - 5s 1ms/step - loss: 0.0748 - val_loss: 0.0856
Epoch 317/1000
3905/3905 [==============================] - 5s 1ms/step - loss: 0.0750 - val_loss: 0.0855
Epoch 318/1000
3905/3905 [==============================] - 5

...

3905/3905 [==============================] - 3s 698us/step - loss: 0.0700 - val_loss: 0.0804
Epoch 463/1000
3905/3905 [==============================] - 3s 702us/step - loss: 0.0671 - val_loss: 0.0769
Epoch 464/1000
3905/3905 [==============================] - 3s 704us/step - loss: 0.0659 - val_loss: 0.0779
Epoch 465/1000
3905/3905 [==============================] - 3s 697us/step - loss: 0.0661 - val_loss: 0.0773
Epoch 466/1000
3905/3905 [==============================] - 3s 701us/step - loss: 0.0661 - val_loss: 0.0771
Epoch 467/1000
3905/3905 [==============================] - 3s 701us/step - loss: 0.0656 - val_loss: 0.0758
Epoch 468/1000
3905/3905 [==============================] - 3s 707us/step - loss: 0.0665 - val_loss: 0.0757
Epoch 469/1000
3905/3905 [==============================] - 3s 708us/step - loss: 0.0655 - val_loss: 0.0753
Epoch 470/1000
3905/3905 [==============================] - 3s 701us/step - loss: 0.0652 - val_loss: 0.0809
Epoch 471/1000
3905/3905 [=================

Epoch 538/1000
3905/3905 [==============================] - 3s 701us/step - loss: 0.0633 - val_loss: 0.0743
Epoch 539/1000
3905/3905 [==============================] - 3s 704us/step - loss: 0.0618 - val_loss: 0.0743
Epoch 540/1000
3905/3905 [==============================] - 3s 704us/step - loss: 0.0622 - val_loss: 0.0824
Epoch 541/1000
3905/3905 [==============================] - 3s 702us/step - loss: 0.0649 - val_loss: 0.0724
Epoch 542/1000
3905/3905 [==============================] - 3s 696us/step - loss: 0.0615 - val_loss: 0.0735
Epoch 543/1000
3905/3905 [==============================] - 3s 701us/step - loss: 0.0631 - val_loss: 0.0717
Epoch 544/1000
3905/3905 [==============================] - 3s 706us/step - loss: 0.0607 - val_loss: 0.0717
Epoch 545/1000
3905/3905 [==============================] - 3s 705us/step - loss: 0.0604 - val_loss: 0.0721
Epoch 546/1000
3905/3905 [==============================] - 3s 698us/step - loss: 0.0600 - val_loss: 0.0746
Epoch 547/1000
3905/3905 [==

Epoch 614/1000
3905/3905 [==============================] - 3s 824us/step - loss: 0.0560 - val_loss: 0.0677
Epoch 615/1000
3905/3905 [==============================] - 3s 753us/step - loss: 0.0598 - val_loss: 0.0817
Epoch 616/1000
3905/3905 [==============================] - 3s 762us/step - loss: 0.0620 - val_loss: 0.0697
Epoch 617/1000
3905/3905 [==============================] - 3s 775us/step - loss: 0.0576 - val_loss: 0.0697
Epoch 618/1000
3905/3905 [==============================] - 3s 842us/step - loss: 0.0551 - val_loss: 0.0680
Epoch 619/1000
3905/3905 [==============================] - 3s 809us/step - loss: 0.0538 - val_loss: 0.0690
Epoch 620/1000
3905/3905 [==============================] - 3s 855us/step - loss: 0.0544 - val_loss: 0.0692
Epoch 621/1000
3905/3905 [==============================] - 3s 891us/step - loss: 0.0533 - val_loss: 0.0678
Epoch 622/1000
3905/3905 [==============================] - 4s 911us/step - loss: 0.0533 - val_loss: 0.0658
Epoch 623/1000
3905/3905 [==

3905/3905 [==============================] - 3s 834us/step - loss: 0.0496 - val_loss: 0.0661
Epoch 691/1000
3905/3905 [==============================] - 4s 1ms/step - loss: 0.0500 - val_loss: 0.0662
Epoch 692/1000
3905/3905 [==============================] - 3s 725us/step - loss: 0.0496 - val_loss: 0.0659
Epoch 693/1000
3905/3905 [==============================] - 3s 709us/step - loss: 0.0493 - val_loss: 0.0668
Epoch 694/1000
3905/3905 [==============================] - 4s 939us/step - loss: 0.0503 - val_loss: 0.0663
Epoch 695/1000
3905/3905 [==============================] - 4s 1ms/step - loss: 0.0513 - val_loss: 0.0661
Epoch 696/1000
3905/3905 [==============================] - 4s 908us/step - loss: 0.0496 - val_loss: 0.0672
Epoch 697/1000
3905/3905 [==============================] - 3s 867us/step - loss: 0.0498 - val_loss: 0.0664
Epoch 698/1000
3905/3905 [==============================] - 4s 939us/step - loss: 0.0523 - val_loss: 0.0659
Epoch 699/1000
3905/3905 [=====================

3905/3905 [==============================] - 3s 821us/step - loss: 0.0471 - val_loss: 0.0719
Epoch 767/1000
3905/3905 [==============================] - 3s 817us/step - loss: 0.0480 - val_loss: 0.0656
Epoch 768/1000
3905/3905 [==============================] - 3s 815us/step - loss: 0.0463 - val_loss: 0.0658
Epoch 769/1000
3905/3905 [==============================] - 3s 821us/step - loss: 0.0472 - val_loss: 0.0663
Epoch 770/1000
3905/3905 [==============================] - 3s 811us/step - loss: 0.0465 - val_loss: 0.0692
Epoch 771/1000
3905/3905 [==============================] - 3s 817us/step - loss: 0.0458 - val_loss: 0.0651
Epoch 772/1000
3905/3905 [==============================] - 3s 819us/step - loss: 0.0454 - val_loss: 0.0663
Epoch 773/1000
3905/3905 [==============================] - 3s 810us/step - loss: 0.0462 - val_loss: 0.0644
Epoch 774/1000
3905/3905 [==============================] - 3s 817us/step - loss: 0.0461 - val_loss: 0.0652
Epoch 775/1000
3905/3905 [=================

In [47]:
# get the Pearson score for biLSTM attention
model = get_model_biLSTM_attention(initializer, LSTM_size)
folder_path = 'TEM-1/model_and_checkpoints/' + f'biLSTM attention LSTM size - {LSTM_size} epochs - {n_epochs} adam_lr 0.003 initializer - {initializer}'
model.load_weights(folder_path + '/weights.995-0.06408.hdf5')

y_pred = model.predict(X_val)
y_pred = np.reshape(y_pred, (-1,))

from scipy.stats import pearsonr, spearmanr
pearson_score = pearsonr(y_val, y_pred)
print(f'Pearson correlation - {pearson_score}')

Pearson correlation - (0.5712665491120772, 5.854758008654332e-39)


In [20]:
folder_path = 'Ubi/model_and_checkpoints/' + f'biLSTM attention LSTM size - {LSTM_size} epochs - {n_epochs} adam_lr {lr} initializer - {initializer}'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


In [21]:
model.save(folder_path + ' full model.hdf5')
with open(folder_path + ' full model.json', 'w') as file:
    file.write(model.to_json())

In [19]:
# get Pearson score for LSTM attention
model = get_model_LSTM_attention('he_uniform', LSTM_size)
model.load_weights('Ubi/model_and_checkpoints/LSTM attention LSTM size - 10 epochs - 1000 adam_lr 0.001 initializer - he_uniform/' + 'weights.672-0.01749.hdf5')

y_pred = model.predict(X_val)
y_pred = np.reshape(y_pred, (-1,))

from scipy.stats import pearsonr, spearmanr
pearson_score = pearsonr(y_val, y_pred)
print(f'Pearson correlation - {pearson_score}')

OSError: Unable to open file (unable to open file: name = 'Ubi/model_and_checkpoints/LSTM attention LSTM size - 10 epochs - 1000 adam_lr 0.001 initializer - he_uniform/weights.672-0.01749.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [27]:
folder_path = 'Ubi/model_and_checkpoints/' + f'LSTM attention LSTM size - {LSTM_size} epochs - {n_epochs} adam_lr {lr} initializer - {initializer}'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


In [28]:
model.save(folder_path + ' full model.hdf5')
with open(folder_path + ' full model.json', 'w') as file:
    file.write(model.to_json())